In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 651, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (233/233), done.
remote: Total 651 (delta 175), reused 50 (delta 37), pack-reused 380 (from 3)
Receiving objects: 100% (651/651), 484.29 KiB | 6.73 MiB/s, done.
Resolving deltas: 100% (385/385), done.


In [2]:
import sys
import pathlib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import yfinance as yf

In [3]:
np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [5]:
from src.data.loaders import (
    get_tickers_history,
    get_return_data,
    get_excess_return,
    get_vix_data,
    get_spread_z,
    get_sector_map,
    get_adv_dollar,
    get_single_ticker_history
)

# Find Tickers with the Most Headlines

In [6]:
from src.data.universe import tickers_with_most_headlines

In [7]:
news_df = pd.read_csv("/kaggle/input/nasdaq-news/nasdaq_news.csv") 

In [8]:
#limit start and stop times
start = pd.Timestamp('2010-01-04 05:00:00+0000', tz='UTC')
end   = pd.Timestamp('2018-12-28 05:00:00+0000', tz='UTC')

#limit news to start and stop times
news_df['Date'] = pd.to_datetime(list(news_df['Date']), utc=True)
news_df = news_df[news_df['Date'] >= start]
news_df = news_df[news_df['Date'] <= end]

#find 300 tickers with the most headlines
tickers = tickers_with_most_headlines(news_df, str(start), str(end), 300)

# Get Price Data

In [9]:
#get yfinance ticker history for all tickers in tickers df
#yfinance will produce the "possibly delisted" message for tickers without information
df = get_tickers_history(list(tickers['ticker']), start=start, end=end)

$X: possibly delisted; no timezone found
$DISH: possibly delisted; no timezone found
$WBA: possibly delisted; no timezone found
$FL: possibly delisted; no timezone found
$SPWR: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00) (Yahoo error = "Data doesn't exist for startDate = 1262581200, endDate = 1546059600")
$BRK: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00)
$DFS: possibly delisted; no timezone found
$PXD: possibly delisted; no timezone found
$AI: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00) (Yahoo error = "Data doesn't exist for startDate = 1262581200, endDate = 1546059600")
$MRO: possibly delisted; no timezone found
$AMTD: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00) (Yahoo error = "Data doesn't exist for startDate = 1262581200, endDate = 1546059600")
$BIG: possibly del

In [10]:
#limit df to only 200 tickers and tickers with data
keep_tickers = list(df['ticker'].drop_duplicates()[:200])
df = df[df['ticker'].isin(keep_tickers)]
tickers = tickers[tickers['ticker'].isin(keep_tickers)]

In [11]:
# limit news to top 200 tickers
news_df = news_df[news_df['Stock_symbol'].isin(keep_tickers)]

In [12]:
def get_date_range(df: pd.DataFrame) -> tuple:
    grouped_by_date = df.groupby(["ticker"]).agg(['min', 'max', 'count'])["Date"]
    start = grouped_by_date["min"].min()
    end = grouped_by_date["max"].max()
    return start, end

In [13]:
#df = get_return_data("/kaggle/input/news-trading/return_data.csv")
#start, end = get_date_range(df)

# Add excess return

In [14]:
def add_excess_return(df, start, end):
    excess_return_df = get_excess_return(df, start, end)
    df = df.merge(excess_return_df, on=["ticker", "Date"], how="left")
    return df

In [15]:
df = add_excess_return(df, start, end)

# Add market regime VIX z-score

In [16]:
def add_vix_z(df, start, end):
    vix_z_df = get_vix_data(start, end)
    format_str = "%Y-%m-%d"
    vix_z_df["Date"] = vix_z_df["Date"].dt.strftime(format_str)
    df["Date"] = df["Date"].dt.strftime(format_str) 
    df = df.merge(vix_z_df, on=["Date"], how="left")
    df['Date'] = pd.to_datetime(df['Date'], utc=True)
    return df

In [17]:
df = add_vix_z(df, start, end)

Yay!🥳


# Add spread z-score

In [18]:
def add_spread_z(existing_df: pd.DataFrame, buffer_days=380) -> pd.DataFrame:
    """
    Use existing OHLCV df, pull buffered history, compute young-safe spread_z on the combined
    Then merge back only the target window rows to prevent nulls.
    """
    df = existing_df.copy()
    start, end = df["Date"].min(), df["Date"].max()

    tickers = sorted(df['ticker'].unique())
    fetch_start = start - timedelta(days=buffer_days)
    fetch_end   = end

    # You already have get_tickers_history(tickers, start, end)
    hist = get_tickers_history(tickers, fetch_start, fetch_end)
    hist["Date"] = pd.to_datetime(hist["Date"], utc=True)

    # Combine buffer + existing; keep existing rows on overlap
    combined = pd.concat([hist, df], ignore_index=True)
    combined = combined.sort_values(['ticker', "Date"])
    combined = combined.drop_duplicates(subset=['ticker', "Date"], keep="last")

    # Compute young-safe spread_z on the full combined range
    combined = get_spread_z(combined)

    # Merge only computed columns back to target window
    cols_to_merge = ['ticker', 'Date', "spread_z"]
    out = df.merge(combined[cols_to_merge], on=['ticker', 'Date'], how="left")

    # Final minimal, causal clean-up to guarantee NON-NULL spread_z in target window:
    # 1) per-ticker forward-fill (past only), 2) same-day cross-section median, 3) final 0
    out["spread_z"] = (
        out.groupby('ticker')["spread_z"].ffill()
           .fillna(out.groupby('Date')["spread_z"].transform("median"))
           .fillna(0.0)
    ).clip(-3, 3)

    return out

In [19]:
df = add_spread_z(df)

# Add sector

In [20]:
def add_sector(df):
    tickers = df["ticker"].unique()
    sector_map = get_sector_map(tickers)
    df = df.join(sector_map, on="ticker")
    return df

In [21]:
df = add_sector(df)

# Add dollar-volume ADV 

In [22]:
def add_adv_dollar(df):
    adv_df = get_adv_dollar(df)
    
    df = df.merge(
        adv_df,
        on=["Date", "ticker"],
        how="left",
    )
    return df

In [23]:
df = add_adv_dollar(df)

# Add Next Day Excess

In [24]:
df["next_day_excess_return"] = df.groupby('ticker')['excess_return'].shift(-1)

# Add 12-1 momentum and momentum rank

In [25]:
def compute_momentum_rank(input_df):
    """
    Compute 12-1 momentum and cross-sectional momentum rank for each ticker.
    12-1 momentum = Close[t-21] / Close[t-252] - 1.
    cross-sectional rank each day = position / N.
    ----------
    Input dataset must contain columns ['ticker', 'Date']; 
    ----------
    Output dataset adds additional columns:['mom_12_1','mom_rank']
    """

    # Fetch data from yfinance starting at a buffer_start date to support 12–1 momentum calculations
    start = input_df['Date'].min() 
    end = input_df['Date'].max() 
    buffer_start = start - timedelta(days=400)
    df = get_tickers_history(list(input_df['ticker'].unique()), buffer_start, end)
    df = df.sort_values(['ticker', 'Date']).reset_index(drop=True)

    # Compute 12-1 momentum
    df['Close_t_21']  = df.groupby('ticker')['Close'].shift(21)
    df['Close_t_252'] = df.groupby('ticker')['Close'].shift(252)
    df['mom_12_1'] = df['Close_t_21'] / df['Close_t_252'] - 1

    # Compute cross-sectional 12-1 momentum ranking
    df['mom_position'] = (
        df.groupby('Date')['mom_12_1']
        .rank(method='first')
    )
    df['N'] = df.groupby('Date')['ticker'].transform('count')
    df['mom_rank'] = df['mom_position'] / df['N']

    # Drop buffer dates data; Keep mom_12_1 and mom_rank columns
    df2 = df[df['Date']>=start]
    df2 = df2[['Date','ticker','mom_12_1','mom_rank']]
    
    # Merge 12-1 momentum and rank into the input dataset
    output_df = pd.merge(input_df, df2, on =['ticker','Date'], how='left')
    return output_df

In [26]:
df = compute_momentum_rank(df)

# Add log_mkt_cap

In [27]:
def get_log_mktcap(input_df):
    """
    Compute log(market capitalization) for each ticker.
    log(market cap_t) = log(Close_t) * SharesOutstanding. 
    * Note that Yahoo only provides the latest shares outstanding, so we don't have the historical 
    shares outstanding at time t. The calculation method for market capitalization may not be accurate.
    ----------
    Input dataset must contain columns ['ticker', 'Date']; 
    ----------
    Output dataset adds additional columns:['log_mktcap']
    """
    
    start = input_df['Date'].min() 
    end = input_df['Date'].max()
    tickers = list(input_df['ticker'].unique())
    
    rows = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)

        # Get CURRENT shares outstanding
        shares = stock.info.get("sharesOutstanding", None)

        # Fetch historical price
        df_price = get_single_ticker_history(ticker, start, end)

        # Add ticker column
        df_price["ticker"] = ticker
        df_price = df_price.reset_index()

        # Compute market cap and log market cap
        if shares is not None:
            df_price["mktcap"] = df_price["Close"] * shares
            df_price["log_mktcap"] = (df_price["mktcap"]).apply(
                lambda x: None if pd.isna(x) else np.log(x)
            )
        else:
            df_price["market_cap"] = None
            df_price["log_mktcap"] = None

        rows.append(df_price)
    # concat all tickers and merge with the input dataset    
    mktcap = pd.concat(rows, ignore_index=True)[['Date','ticker','log_mktcap']]
    output_df = pd.merge(input_df, mktcap, on=['Date','ticker'], how='left')
    
    return output_df

In [28]:
df = get_log_mktcap(df)

/tmp/ipykernel_320/3068293802.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mktcap = pd.concat(rows, ignore_index=True)[['Date','ticker','log_mktcap']]


# Add SPY r1

In [29]:
def compute_SPY_r1 (input_df):
    """
    Compute SPY r1 = (Close_t/Clost_t_1) -1
    ----------
    Input dataset must contain columns ['Date']. 
    ----------
    Output dataset adds additional columns:['spy_r1']
    """
    start = input_df['Date'].min() 
    end = input_df['Date'].max()
    buffer_start = start - timedelta(days=7)
    
    df_spy = get_single_ticker_history("SPY", buffer_start, end)
    df_spy["spy_r1"] = df_spy["Close"] / df_spy["Close"].shift(1) - 1
    df_spy = df_spy[df_spy['Date']>=start]
    df_spy = df_spy[['Date','spy_r1']]
    
    output_df = pd.merge(input_df, df_spy, on=['Date'], how='left')
    return output_df

In [30]:
df = compute_SPY_r1(df)

# Add News Flag

In [31]:
def count_headlines_all_days(news_df, start, end):
    """Counts the number of headlines for each ticker symbol each day
    Input: news_df pandas dataframe with ticker column for ticker symbols and date for the headline date
    Output: pandas dataframe containing the number of headlines per ticker per day
                indexes are dates in string and tickers as the column names"""
    
    #check columns in dataframe
    columns = list(news_df.columns)
    if (('date' not in columns) and ('Date' not in columns)) or (('ticker' not in columns) and ('Stock_symbol' not in columns)):
        print('input dataframe does not have both ticker and date columns')
        return pd.DataFrame()
    
    #find column names
    date_col = 'date' if 'date' in columns else 'Date'
    ticker_col = 'ticker' if 'ticker' in columns else 'Stock_symbol'
    
    # Count occurrences in the date column
    headline_dates = news_df[date_col]#.str[:10]#.value_counts()
    df = pd.DataFrame({ticker_col: news_df[ticker_col],
                       date_col: headline_dates})
    
    # count headlines per day per ticker
    df = df.groupby([date_col, ticker_col]).size().unstack(fill_value=0)
    
    #create list of dates needed
    format_code = "%Y-%m-%d"# Corresponds to 'YYYY-MM-DD'
    set_of_dates = set(df.index)
    date_min = start#datetime.strptime(min('2010-01-04',min(set_of_dates)), format_code).date() #datetime(2000,1,1).date()#
    date_max = end #datetime.strptime(max('2018-12-28',max(set_of_dates)), format_code).date()
    date_lst = [(date_min+timedelta(i)) for i in range(int((date_max-date_min).days)+1)]
    
    #find dates not in dataframe
    missing_dates = dict([(day,int(0)) for day in set(date_lst).difference(set(df.index))])
    
    #add missing dates to dataframe
    tickers = list(set(df.columns))
    tickers.sort()
    empty_dict = dict([(ticker, missing_dates) for ticker in tickers])
    add_dates = pd.DataFrame(empty_dict)
    df = pd.concat([df, add_dates], ignore_index=False)
    
    #sort rows and columns
    df = df.sort_index()
    df = df.T
    df = df.sort_index()
    
    return df

In [32]:
#from src.data.features.news_features import count_headlines_all_days


def add_news_flag(news_df, price_df, start=None, end=None):
    """adds a new news flag column: 0=no news, 1=news
    input: news_df with 'date', 'ticker', and other columns
            price_df with 'Date', 'ticker', and other columns
            optional start and end Timestamps
    output: dataframe df
    """

    #count headlines per ticker per day
    news_count = count_headlines_all_days(news_df, start, end)
    
    #filter count_df by date
    news_count = news_count.T
    news_count['date'] = pd.to_datetime(list(news_count.index), utc=True)
    if start is not None: news_count = news_count[news_count['date'] >= start]
    if end is not None: news_count = news_count[news_count['date'] <= end]
    
    #convert news_count df to different format
    news_cols = list(news_count.columns)
    news_count = news_count.melt(id_vars=['date'], value_vars=news_cols, 
                  var_name='ticker', value_name='news flag')

    # change count to flag: 0=no news, 1=news
    news_count['news flag'] = [flag if flag < 2 else 1 for flag in news_count['news flag']]
    news_count['date'] = pd.to_datetime(news_count['date'], utc=True)
    news_count.sort_values(['date','ticker'], inplace=True)
    
    #add news flag: 0=no news, 1=news
    price_df = pd.merge(price_df, news_count, left_on=['Date','ticker'], 
              right_on=['date','ticker'])

    return price_df

In [33]:
df = add_news_flag(news_df, df, start=start, end=end)

# Split Data

In [34]:
df['split'] = 'train'
df.loc[df['Date'] >="2015-01-01", "split"] = "val"
df.loc[df['Date'] >= "2017-01-01", "split"] = 'test'

# Final Complete Dataset

In [35]:
df.to_csv('final_dataset.csv', index=False)

In [36]:
news_df.to_csv('final_news_dataset.csv', index=False)

In [37]:
tickers.to_csv('top_tickers.csv', index=False)

In [38]:
get_return_data("/kaggle/working/final_dataset.csv")

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,Capital Gains,...,dollar_volume,adv_dollar,next_day_excess_return,mom_12_1,mom_rank,log_mktcap,spy_r1,date,news flag,split
0,2010-01-05 00:00:00+00:00,53.698790,54.428207,53.664059,53.941929,13469263.0,0.0,0.0,GE,NaN,...,7.265580e+08,NaN,-0.006825,NaN,NaN,NaN,NaN,2010-01-05 00:00:00+00:00,0,train
1,2010-01-07 00:00:00+00:00,53.768234,57.241638,53.594568,56.442753,38701038.0,0.0,0.0,GE,NaN,...,2.184393e+09,NaN,0.011810,NaN,NaN,NaN,NaN,2010-01-07 00:00:00+00:00,0,train
2,2010-01-08 00:00:00+00:00,56.651150,57.971039,56.512210,57.658432,24019636.0,0.0,0.0,GE,NaN,...,1.384935e+09,NaN,-0.001118,NaN,NaN,NaN,NaN,2010-01-08 00:00:00+00:00,0,train
3,2010-01-12 00:00:00+00:00,57.588993,58.492077,57.554262,58.248943,13484329.0,0.0,0.0,GE,NaN,...,7.854479e+08,NaN,-0.001703,NaN,NaN,NaN,NaN,2010-01-12 00:00:00+00:00,0,train
4,2010-01-13 00:00:00+00:00,58.214235,58.769981,57.554291,58.457375,13634482.0,0.0,0.0,GE,NaN,...,7.970360e+08,NaN,-0.009203,NaN,NaN,NaN,NaN,2010-01-13 00:00:00+00:00,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437416,2018-12-21 00:00:00+00:00,17.770000,17.950001,16.850000,16.959999,9401000.0,0.0,0.0,UAA,NaN,...,1.594410e+08,1.472916e+08,0.020259,NaN,NaN,NaN,NaN,2018-12-21 00:00:00+00:00,1,test
437417,2018-12-24 00:00:00+00:00,16.740000,17.360001,16.570000,16.750000,2688600.0,0.0,0.0,UAA,NaN,...,4.503405e+07,1.477448e+08,0.018078,NaN,NaN,NaN,NaN,2018-12-24 00:00:00+00:00,0,test
437418,2018-12-26 00:00:00+00:00,16.790001,17.840000,16.520000,17.820000,3966400.0,0.0,0.0,UAA,NaN,...,7.068125e+07,1.478580e+08,-0.003903,NaN,NaN,NaN,NaN,2018-12-26 00:00:00+00:00,1,test
437419,2018-12-27 00:00:00+00:00,17.580000,17.930000,17.100000,17.910000,4360800.0,0.0,0.0,UAA,NaN,...,7.810193e+07,1.476052e+08,-0.016623,NaN,NaN,NaN,NaN,2018-12-27 00:00:00+00:00,0,test


In [39]:
!rm -rf /kaggle/working/multimodal-eq-sizing